In [2]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
  
ProjectFolderName = 'mired_rata-ainhoa-2021-11-17' #cambiar por nombre de la carpeta
VideoType = 'mp4' 

#don't edit these:
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

In [ ]:
#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"

In [ ]:
import deeplabcut
deeplabcut.__version__

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before: 
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file

# CREATING A TRAINING DATASET

In [ ]:
#check the docstring for full options you can do!
#num_shuffles --> If the user wishes to benchmark the performance of the DeepLabCut, they can create multiple
#training datasets by specifying an integer value to the num_shuffles.
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug', num_shuffles=1)
#cada it de creation_dataset crea un archivo .mat y .pickle (contiene meta info).
#se crean tambien subdirectorios train y test con pose_cfg.yaml (editar el de train andtes de empezar funcion training)
 

# TRAINING THE NETWORK

In [ ]:
#Tiempo estimado:  NVIDIA GeForce GTX 1080 Ti GPU, it takes ≈ 6 hrs to train the network for at least 200,000 iterations 
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.

#shuffle: Integer value specifying the shuffle index to select for training. Default is set to 1
#trainingsetindex: Integer specifying which Training set Fraction to use. By default the first (note that Training Fraction is a list in config.yaml).
#gputouse: Natural number indicating the number of your GPU (see number in nvidia-smi). If you do not have a GPU put None
#max_snapshots_to_keep: This sets how many snapshots are kept, i.e. states of the trained network. Every saving iteration a snapshot is stored, however only the last max_snapshots_to_keep many are kept! If you change this to None, then all are kept.
#autotune: property of TensorFlow, tested to be faster if set to ’False’ (default)
#displayiters: this variable is actually set in pose_config.yaml. However, you can overwrite it with this. If None, the value from pose_config.yaml is used, otherwise it is overwritten! Default: None
#saveiters: this variable is set in pose_config.yaml. However, you can overwrite it with this. If None, the value from there is used, otherwise it is overwritten! Default: None
#batch_size: (default 1) This specifies how many frames to process at once during inference

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100,saveiters=500)
#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....
#weights de TF en subdirect \models\pre-trained\Pose_Estimation_Tensorflow

If the user wishes to restart the training at a specific
checkpoint they can specify the full path of the checkpoint to the variable ‘init_weights’ in the pose_cfg.yaml file
under the ‘train’ subdirectory 
#init_weights: /content/drive/MyDrive/mired_rata-ainhoa-2021-11-17/dlc-models/iteration-1/mired_rataNov17-trainset95shuffle1/train/snapshot-120 (por ejemplo)


# EVALUATING THE NETWORK
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .csv file in a subdirectory under evaluation-results
The plots can be customized by editing the config.yaml file (i.e. the colormap, scale, marker size (dotsize), and
transparency of labels (alphavalue) can be modified).
Note that by default the human labels are
plotted as plus (‘+’), DeepLabCut’s predictions either as ‘.’ (for confident predictions with likelihood > p-cutoff) and
’x’ for (likelihood <= p-cutoff).

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,shuffle=[1],plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

# # OPTIONAL REFINEMENT STEP (OUTSIDE COLAB)
## There is an optional refinement step you can do outside of Colab:
If your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
Please see the repo and protocol instructions on how to refine your data!

# VIDEO ANALYSIS
The user needs to first choose a checkpoint with the best
evaluation results for analyzing the videos. In this case, the user can enter the corresponding index of the checkpoint
to the variable snapshotindex in the config.yaml file. By default, the most recent checkpoint (i.e. last) is used for
analyzing the video. 

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType, shuffle=1, save_as_csv=True)

The labels are stored in a MultiIndex Pandas Array ([41], http://pandas.pydata.org), which contains the name
of the network, body part name, (x, y) label position in pixels, and the likelihood for each frame per body part.
 if the flag save_as_csv is set to True, the data can also be exported in comma-separated values format
(.csv), which in turn can be imported in many programs, such as MATLAB, R, Prism, etc.

# CREATE LABELED VIDEO
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
#editar dotsize label en config.yaml
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)